# 多重共线性

**多重共线性是指多元回归模型中有两个或两个以上的自变量X**，它们之间具有高度的相关性。当某些特征高度相关时，我们可能很难区分它们对因变量的个体影响。

## 方差膨胀因子

在VIF方法中，我们选择每个特征并将其与所有其他特征进行回归。对于每个回归，因子计算如下：
$$VIF = 1/(1-R^2)$$
其中，R平方是线性回归中的决定系数。它的值介于0和1之间。

正如我们从公式中看到的，R平方的值越大，VIF越大。因此，VIF越大，相关性越强。这与较高的R平方值表示较强的共线性的事实一致。通常，VIF高于5表示高多重共线性。

statsmodels提供了一个名为variance_inflation_factor()的函数来计算VIF。

语法：statmodels.stats.outliers_influence.variance_inflation_factor（exog，exog_idx）  
主要参数：  
exog：一个数组，包含对其执行线性回归的特征。  
exog_idx：要测量其对其他特征的影响的附加特征的索引。


In [14]:
import pandas as pd  
  
# the dataset   
data = pd.read_csv('../data/BMI.csv') 
  
# printing first few rows 
print(data.head())

   Gender  Height  Weight  Index
0    Male     174      87      4
1    Male     174      87      2
2  Female     174      87      4
3  Female     174      87      3
4    Male     174      87      3


In [2]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 

# creating dummies for gender 
data['Gender'] = data['Gender'].map({'Male':0, 'Female':1}) 

# the independent variables set 
X = data[['Gender', 'Height', 'Weight']] 

# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 

# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))] 

print(vif_data)

  feature           VIF
0  Gender      1.090909
1  Height  17557.511404
2  Weight  17621.035960


正如我们所看到的，身高和体重具有非常高的VIF值，表明这两个变量高度相关。这是预料之中的，因为一个人的身高确实会影响他们的体重。因此，将这两个特征一起考虑会导致具有高多重共线性的模型。

其实方差因子通常成对出现，通常较大的两个方差膨胀因子表明这两个变量自身本来就具有高度相关性，即这两个变量一起才解释一种东西，可以考虑去除一个自己并不怎么感兴趣的变量即可，也回到了我们最开始的目的：多重线性回归模型的主要假设之一是我们的预测变量彼此不相关，我们希望预测变量与反应变量相关，而不是彼此之间具有相关性。

## 相关系数

In [15]:
data = data[['Height', 'Weight']]
p_value = data.values

In [20]:
value = np.corrcoef(p_value)

In [21]:
np.argwhere(value == 1)

array([[0, 0],
       [0, 1],
       [0, 2],
       [0, 3],
       [0, 4],
       [1, 0],
       [1, 1],
       [1, 2],
       [1, 3],
       [1, 4],
       [2, 0],
       [2, 1],
       [2, 2],
       [2, 3],
       [2, 4],
       [3, 0],
       [3, 1],
       [3, 2],
       [3, 3],
       [3, 4],
       [4, 0],
       [4, 1],
       [4, 2],
       [4, 3],
       [4, 4]], dtype=int64)

## 处理方法

### 手动剔除变量

手动剔除变量，最简单的方法就是对存在共线性的自变量进行一定的筛选。将VIF值最大的两个自变量进行剔除，再次分析。

### 逐步回归

逐步回归法是在模型中逐个引入自变量，自动筛选出有影响的变量。逐步回归根据模型统计意义的检验以及调整后R方的变化情况来判断新引入的变量是否引起共线性。如果新引入的变量使得模型统计意义检验能通过，且又能提高模型的调整后R方，则应该引入，否则不引入。

### 岭回归

岭回归是利用岭估计来代替普通最小二乘估计，以损失部分信息为代价来寻找效果稍差但回归系数更符合实际情况的模型方程。当自变量之间存在多重共线性，岭回归是相对比较稳定的方法，且岭回归估计的回归系数的标准差也较小。

在现实中，真正应用来选择正则化系数的技术是交叉验证，并且选择的标准非常明确——选择让交叉验证下的均方误差最小的正则化系数α。

### 增大样本量

在建立回归模型时，如果变量的样本数据太少，很容易产生多重共线性问题。所以可以通过增大样本量的方法，克服多重共线性。但是在实际研究中是不现实的，因为我们没有办法确定增加什么多少样本才能克服多重共线性，也有可能在增加了样本量的同时，产生了一些新的问题，导致模型的拟合变差，没有达到我们所期望的效果。